<h1>best running</h1>

link to kaggle

In [1]:
! gdown https://drive.google.com/uc?id=1AbxjCBByF-xGV3c7FJ1ztZC1HO0O0uJ0

Downloading...
From: https://drive.google.com/uc?id=1AbxjCBByF-xGV3c7FJ1ztZC1HO0O0uJ0
To: /content/kaggle.json
100% 66.0/66.0 [00:00<00:00, 223kB/s]


In [2]:
! pip install kaggle
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'
! kaggle competitions download -c comp-551-fall-2021
! unzip images_l.pkl.zip
! unzip images_test.pkl.zip
! unzip images_ul.pkl.zip
! unzip labels_l.pkl.zip

  0% 0.00/141k [00:00<?, ?B/s]
100% 141k/141k [00:00<00:00, 68.9MB/s]
 99% 194M/197M [00:00<00:00, 215MB/s]
100% 197M/197M [00:00<00:00, 218MB/s]
 92% 91.0M/99.0M [00:00<00:00, 224MB/s]
100% 99.0M/99.0M [00:00<00:00, 223MB/s]
 91% 180M/198M [00:00<00:00, 235MB/s]
100% 198M/198M [00:00<00:00, 218MB/s]
Archive:  images_l.pkl.zip
  inflating: images_l.pkl            
Archive:  images_test.pkl.zip
  inflating: images_test.pkl         
Archive:  images_ul.pkl.zip
  inflating: images_ul.pkl           
Archive:  labels_l.pkl.zip
  inflating: labels_l.pkl            


imports

In [3]:
import io
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU, ZeroPadding2D,Convolution2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import History
from tensorflow.keras.optimizers import Nadam, Adam, SGD
from keras.applications.vgg16 import VGG16
from keras.applications.densenet import DenseNet169
from keras.applications.resnet import ResNet152, ResNet101, ResNet50
import matplotlib.pyplot as plt
from keras.utils import np_utils
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras import layers, regularizers, Input
from keras.layers import *
from keras.callbacks import ReduceLROnPlateau
from keras import optimizers
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from google.colab import drive
import h5py
import random
import torch

load datasets

In [4]:
# Image data for training (30,000 sample, each sample is a 56x56 image)
# train_labels = pd.read_csv('train_max_y.csv')
# train_images = pd.read_pickle('train_max_x')
# test_images = pd.read_pickle('test_max_x')
with open("images_l.pkl", 'rb') as f:
    # load into as a numpy array
    train_labelled_data = pickle.load(f)
# Test images. The prediction corresponding to these images should be uploaded. (15,000 samples)
with open("images_test.pkl", 'rb') as f:
    test_data = pickle.load(f)
# Labels for training (30,000 rows, each row is a size 36 binary vector, which is the label to the corresponding image)
with open("labels_l.pkl", 'rb') as f:
    train_labelled_y = pickle.load(f)
# Additional images that can be used for training the classifier.
# Labels for these images are not provided. (30,000 samples, where each sample is a 56x56 image)
with open("images_ul.pkl", 'rb') as f:
    train_unlabelled_x = pickle.load(f)

# Pre processing

In [5]:
# load process data
! gdown https://drive.google.com/uc?id=1wyUpGYONbWRjdaLgeiEH2ryqm_kMOMF_
! gdown https://drive.google.com/uc?id=1tFt0ddczfNPmq8IUNnZ8ifGUbgzP9DxU
! gdown https://drive.google.com/uc?id=1bOq70mP5ghxfiFtbK3rTbPDcefkxgyQx

Downloading...
From: https://drive.google.com/uc?id=1wyUpGYONbWRjdaLgeiEH2ryqm_kMOMF_
To: /content/processed_train_set.csv
100% 386M/386M [00:02<00:00, 176MB/s]
Downloading...
From: https://drive.google.com/uc?id=1tFt0ddczfNPmq8IUNnZ8ifGUbgzP9DxU
To: /content/processed_test_set.csv
100% 193M/193M [00:01<00:00, 186MB/s]
Downloading...
From: https://drive.google.com/uc?id=1bOq70mP5ghxfiFtbK3rTbPDcefkxgyQx
To: /content/processed_ul_train_set.csv
100% 386M/386M [00:02<00:00, 192MB/s]


In [6]:
# load models
# ! gdown https://drive.google.com/uc?id=1gpLzFm3QW1q__KMAg4QWe2pqVveAmd_y
# ! gdown https://drive.google.com/uc?id=1DLNI6xb1SNEFMcEcNgJb_q0WYR5OlKi2
# ! gdown https://drive.google.com/uc?id=1VFcPDEt_Ges0eJ7FB9iKoWV4goSMGqk2
# ! gdown https://drive.google.com/uc?id=1p_etGMx0I57VG6SGR_XBp3y153gNLHNR
# ! gdown https://drive.google.com/uc?id=1nO0-eQyOBKSe-VqK3jwQ6iCFy-tkf6xK
# ! gdown https://drive.google.com/uc?id=1v8MLZ69Py9cZmi9ippWBwMc7tqnfKNlv
# ! gdown https://drive.google.com/uc?id=1df0K-MQQ3c1VEAOSUqNq931La8sClpxp

! gdown https://drive.google.com/uc?id=1RoQyzuzt88rBnctuWZRoPZMlyJvHsofU
! gdown https://drive.google.com/uc?id=1v448DaA5Zx9xdM0-mlmOAo-BmSIy5rwC
! gdown https://drive.google.com/uc?id=1VdJ8BMOnrSb4kyBewHZqg_OUjMVgy6XA
! gdown https://drive.google.com/uc?id=1VQ2q8-aTFntdLMOllG-4mIHexwC_Mnma
! gdown https://drive.google.com/uc?id=1REQfSvI-Grz6TgGe35KXbv7LmtiwwYxh
! gdown https://drive.google.com/uc?id=12u-rpWk_OoMQnhXq-dl6S1yOBSKjXL-x
! gdown https://drive.google.com/uc?id=1IQ-NqDpJ7RUcRUY6-YLOuiM8zY1WLpL7
! gdown https://drive.google.com/uc?id=18bQmyfKnXd6iD8zWpJ6VKnvfU6xnNwhX
! gdown https://drive.google.com/uc?id=1JSu9bdn8qR5bNOpVlS83KuW7_3SJffny
! gdown https://drive.google.com/uc?id=1rLBQ_OiQ0lFZwvw0vkrFfufGgWyim03z
! gdown https://drive.google.com/uc?id=1K-cuAO6jwHA8HIUilN2YDB6osBuRgSBh
! gdown https://drive.google.com/uc?id=1T4wL3v8RB2uK4JMbC2Lf0DOKfCoN6KI8
! gdown https://drive.google.com/uc?id=1Kx_rd8KoXNlOV5sBJysZXQ8Hgw1Y8aYL
! gdown https://drive.google.com/uc?id=1llVTiVKypK3UlH3lfDIIFzblF4Yesw7Y


! gdown https://drive.google.com/uc?id=1YKExD85CaznZqjKOYPc3of08iCdMrUtz
! gdown https://drive.google.com/uc?id=186i8-O8q07Q2CrEz7O2GXaj9Yz9NKXwC
! gdown https://drive.google.com/uc?id=1-IIib1GFCDFYvtvCoME98P8VszmC9yQT
! gdown https://drive.google.com/uc?id=1ZdqmrMxnCz6NbJkYd5DfmGdQ_34t7A95
! gdown https://drive.google.com/uc?id=1cjlv8lHW1PtPdrS6S9-Myb0zaCrLzl8v
! gdown https://drive.google.com/uc?id=1s6RhlU7hHfUE7N1eAKWf6dCbflYP6AKs
! gdown https://drive.google.com/uc?id=1y99YjBiainJY7UW43J0RMoCt6boMeuEp
! gdown https://drive.google.com/uc?id=1mlIrigUbVDuV2NJRRB0TG5W0YHtOGkIw
! gdown https://drive.google.com/uc?id=1BdNUh1oABKxqpLexSVHRocaKmVn11u4y
! gdown https://drive.google.com/uc?id=1_ff2WbG8iY9lMzdvMLIFtIEuqo7FV-sU
! gdown https://drive.google.com/uc?id=1Vrc84KkEJP7YaoxXzVdPOQfXib3lnktD
! gdown https://drive.google.com/uc?id=1uZcZUwGkf-nAPNS8EGC-wGeWeAm8qXOI
! gdown https://drive.google.com/uc?id=1771rkCi9A1bB77ZiOfyOZEsdyOywHf1E
! gdown https://drive.google.com/uc?id=1N5v2Aof5Ht4H4p-1ZvC3QHDBMVJUPMVX
! gdown https://drive.google.com/uc?id=12-wzZpAyipKsefGB4HeUAKDZsvw5tS_N


Downloading...
From: https://drive.google.com/uc?id=1RoQyzuzt88rBnctuWZRoPZMlyJvHsofU
To: /content/model1_valid_97.9_3.pkl
100% 38.2M/38.2M [00:00<00:00, 91.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1v448DaA5Zx9xdM0-mlmOAo-BmSIy5rwC
To: /content/model1_valid_97.8_1.pkl
100% 38.2M/38.2M [00:00<00:00, 82.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1VdJ8BMOnrSb4kyBewHZqg_OUjMVgy6XA
To: /content/model1_valid_97.8_2.pkl
100% 38.2M/38.2M [00:00<00:00, 145MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1VQ2q8-aTFntdLMOllG-4mIHexwC_Mnma
To: /content/model1_valid_97.9_2.pkl
100% 38.2M/38.2M [00:00<00:00, 67.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1REQfSvI-Grz6TgGe35KXbv7LmtiwwYxh
To: /content/model1_valid_97.9_1.pkl
100% 38.2M/38.2M [00:00<00:00, 74.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=12u-rpWk_OoMQnhXq-dl6S1yOBSKjXL-x
To: /content/model1_valid_97.7_3.pkl
100% 38.2M/38.2M [00:00<00:00, 92.1MB/s]
Downloading...
F

In [11]:
# model1_1 = pickle.load(open("model1_valid_97.9_3.pkl", 'rb'))
# model1_2 = pickle.load(open("model1_valid_97.9_2.pkl", 'rb'))
# model1_3 = pickle.load(open("model1_valid_97.9_1.pkl", 'rb'))
# model1_4 = pickle.load(open("model1_valid_97.8_2.pkl", 'rb'))
# model1_5 = pickle.load(open("model1_valid_97.8_1.pkl", 'rb'))
# model1_6 = pickle.load(open("model1_valid_97.7_2.pkl", 'rb'))
# model1_7 = pickle.load(open("model1_valid_97.7_3.pkl", 'rb'))
# model1_8 = pickle.load(open("model1_valid_97.7_1.pkl", 'rb'))
model1_1 = pickle.load(open("model1_valid_99.1_1.pkl", 'rb'))
model1_2 = pickle.load(open("model1_valid_99.0_1.pkl", 'rb'))
model1_3 = pickle.load(open("model1_valid_99.05_1.pkl", 'rb'))
model1_4 = pickle.load(open("model1_valid_98.95_1.pkl", 'rb'))
model1_5 = pickle.load(open("model1_valid_98.85_1.pkl", 'rb'))
# model1_6 = pickle.load(open("model1_valid_98.5_1.pkl", 'rb'))


# model2_1 = pickle.load(open("model2_valid_97.9.pkl", 'rb'))
# model2_2 = pickle.load(open("model2_valid_97.9_1.pkl", 'rb'))
# model2_3 = pickle.load(open("model2_valid_97.5.pkl", 'rb'))
# model2_4 = pickle.load(open("model2_valid_97.3.pkl", 'rb'))
# model2_5 = pickle.load(open("model2.pkl", 'rb'))
# model2_6 = pickle.load(open("model2_valid_97.3_1.pkl", 'rb'))
model2_1 = pickle.load(open("model2_valid_98.1_1.pkl", 'rb'))
model2_2 = pickle.load(open("model2_valid_98.15_1.pkl", 'rb'))
model2_3 = pickle.load(open("model2_valid_98.05_3.pkl", 'rb'))
model2_4 = pickle.load(open("model2_valid_97.95_2.pkl", 'rb'))
model2_5 = pickle.load(open("model2_valid_98.05_1.pkl", 'rb'))
model2_6 = pickle.load(open("model2_valid_98.05_2.pkl", 'rb'))
model2_7 = pickle.load(open("model2_valid_98.0_1.pkl", 'rb'))

# Digit
we divide our training and prediction into two parts: digit and letters.

In [12]:
# load testing data for model testing
x_test=np.array(pd.DataFrame(pd.read_csv('processed_test_set.csv')))

x_test=x_test.reshape((15000,56,56))

for i in range(x_test.shape[0]):
  x_test[i] = (x_test[i]>200).astype('int32')*255
x_test = torch.Tensor(x_test)
x_test = torch.unsqueeze(x_test,dim=3)/255
x_test = x_test.repeat(1,1,1,3)
x_test = x_test.numpy()
x_test = x_test.astype('float32')

In [22]:
with open("images_test.pkl", 'rb') as f:
    test_data = pickle.load(f)
x_test = test_data.reshape(-1, 56, 56, 1)

In [23]:
result_1 = model1_1.predict(x_test,verbose=1)
result_2 = model1_2.predict(x_test,verbose=1)
result_3 = model1_3.predict(x_test,verbose=1)
result_4 = model1_4.predict(x_test,verbose=1)
result_5 = model1_5.predict(x_test,verbose=1)
# result_6 = model1_6.predict(x_test,verbose=1)
# result_7 = model1_7.predict(x_test,verbose=1)
# result_8 = model1_8.predict(x_test,verbose=1)

469/469 [==============================] - 2s 5ms/step


In [ ]:
result = np.argmax(result_1,axis=1)
print(result[0:10])

[0 4 9 0 6 9 3 2 1 6]


average the models prediction to get a more reasonable prediction

In [15]:
# averaging
result = np.zeros((15000,10))
for i in range (0,result_1.shape[0]):
  for j in range (0,result_1.shape[1]):
    # result[i][j] = (result_1[i][j] + result_2[i][j] + result_3[i][j] + result_4[i][j] + result_5[i][j] + result_6[i][j] + result_7[i][j]  )/7
    result[i][j] = (result_1[i][j] + result_2[i][j] + result_3[i][j] + result_4[i][j] + result_5[i][j])/5

result = np.argmax(result, axis = 1)
print(result[0:10])

[0 4 9 0 6 9 3 2 1 6]


vote for mode for models prediction to get a more reasonable prediction

In [24]:
# mode
result_1 = np.argmax(result_1, axis = 1)
result_2 = np.argmax(result_2, axis = 1)
result_3 = np.argmax(result_3, axis = 1)
result_4 = np.argmax(result_4, axis = 1)
result_5 = np.argmax(result_5, axis = 1)
# result_6 = np.argmax(result_6, axis = 1)
# result_7 = np.argmax(result_7, axis = 1)

print(result_1[0:10])
print(result_2[0:10])
print(result_3[0:10])
print(result_4[0:10])
print(result_5[0:10])
# print(result_6[0:10])
# print(result_7[0:10])

from scipy import stats
result = np.zeros(15000)
for i in range (result_1.shape[0]):
  # array = np.array([result_1[i],result_2[i],result_3[i],result_4[i],result_5[i],result_6[i],result_7[i] ])
  array = np.array([result_1[i],result_2[i],result_3[i],result_4[i],result_5[i] ])
  mode = stats.mode(array)
  result[i]=mode[0][0]
result = result.astype(int)
print(result[0:10])

[0 4 9 0 6 9 3 2 1 6]
[0 4 9 0 6 9 3 2 1 6]
[0 4 9 0 6 9 3 2 1 6]
[0 4 9 0 6 9 3 2 1 6]
[0 4 9 0 6 9 3 2 1 6]
[0 4 9 0 6 9 3 2 1 6]


# Letter

In [25]:
with open("images_test.pkl",'rb') as f:
  x_test_data = pickle.load(f)
x_test = x_test_data.reshape(-1,56,56,1)

In [26]:
result2_1 = model2_1.predict(x_test,verbose=1)
result2_2 = model2_2.predict(x_test,verbose=1)
result2_3 = model2_3.predict(x_test,verbose=1)
result2_4 = model2_4.predict(x_test,verbose=1)
result2_5 = model2_5.predict(x_test,verbose=1)
result2_6 = model2_6.predict(x_test,verbose=1)
result2_7 = model2_7.predict(x_test,verbose=1)

469/469 [==============================] - 2s 5ms/step


In [18]:
# averaging
result2 = np.zeros((15000,26))
for i in range (0,result2_1.shape[0]):
  for j in range (0,result2_1.shape[1]):
    result2[i][j] = (result2_1[i][j] + result2_2[i][j] + result2_3[i][j] + result2_4[i][j]+ result2_5[i][j] + result2_6[i][j] + result2_7[i][j] )/7

result2 = np.argmax(result2, axis = 1)
print(result2[0:10])

[19 11  9 11  9 24 14 18 22  6]


In [27]:
# mode
result2_1 = np.argmax(result2_1, axis = 1)
result2_2 = np.argmax(result2_2, axis = 1)
result2_3 = np.argmax(result2_3, axis = 1)
result2_4 = np.argmax(result2_4, axis = 1)
result2_5 = np.argmax(result2_5, axis = 1)
result2_6 = np.argmax(result2_6, axis = 1)
result2_7 = np.argmax(result2_7, axis = 1)

print(result2_1[0:10])
print(result2_2[0:10])
print(result2_3[0:10])
print(result2_4[0:10])
print(result2_5[0:10])
print(result2_6[0:10])
print(result2_7[0:10])

from scipy import stats
result2 = np.zeros(15000)
for i in range (result2_1.shape[0]):
  array = np.array([result2_1[i],result2_2[i],result2_3[i],result2_4[i],result2_5[i],result2_6[i],result2_7[i] ])
  mode = stats.mode(array)
  result2[i]=mode[0][0]
result2 = result2.astype(int)
print(result2[0:10])

[19 11  9 11  9 24 14 18 22  6]
[19 11  9 11  9 24 14 18 22  6]
[19 11  9 11  9 24 14 18 22  6]
[19 11  9 11  9 24 14 18 22  6]
[19 11  9 11  9 24 14 18 22  6]
[19 11  9 11  9 24 14 18 22  6]
[19 11  9 11  9 24 14 18 22  6]
[19 11  9 11  9 24 14 18 22  6]


# output

In [28]:
# transform 0-9 number to string like 0100000000, 0000001000
def transform_digit (num):
    temp_list = np.zeros(10)
    temp_list[num]=1
    temp_list = temp_list.astype(int)
    str1 = ''.join(str(i) for i in temp_list)
    return str1

# tranform 0-25 number to 26-bit string with the index at the num value to be 1
def transform_letter (num):
    temp_list = np.zeros(26)
    temp_list[num] = 1
    
    temp_list = temp_list.astype(int)
    str1 = ''.join(str(i) for i in temp_list)
    return str1

output = list()
for i in range (0,result.shape[0]):
    tempStr = transform_digit(result[i])+transform_letter(result2[i])
    output.append([i,tempStr])

In [29]:
df=pd.DataFrame(output)
df.columns = ['# Id','Category']
df.to_csv('result.csv',index=False)